In [1]:
from transformers import pipeline

In [2]:
classifier = pipeline('sentiment-analysis')
classifier([
    "I've been waiting for EmKa Academy course my whole life",
    "I hate you brother..."
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9860089421272278},
 {'label': 'NEGATIVE', 'score': 0.9979550838470459}]

## What happens inside pipeline function ?

![All Stages](image-2.png)

![Alt Stage 1](image.png)

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for EmKa Academy course my whole life",
    "I hate you brother..."
]

inputs = tokenizer(raw_inputs, padding= True, truncation= True, return_tensors= "pt")
inputs

{'input_ids': tensor([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 7861, 2912, 2914, 2607, 2026,
         2878, 2166,  102],
        [ 101, 1045, 5223, 2017, 2567, 1012, 1012, 1012,  102,    0,    0,    0,
            0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}

![Stage 2](image-1.png)

In [4]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

outputs = model(**inputs)
print(outputs)

print(outputs.last_hidden_state.shape) # batch size, sequence length, hidden size

BaseModelOutput(last_hidden_state=tensor([[[ 0.4751,  0.1254,  0.3595,  ..., -0.0702,  0.1513,  0.1720],
         [ 1.0680,  0.8779, -0.1211,  ...,  0.1018,  0.5267, -0.0016],
         [ 1.1749,  0.0640,  0.0409,  ...,  0.1418, -0.3109, -0.5614],
         ...,
         [ 0.6120,  0.3241,  0.1171,  ..., -0.5674,  0.5678, -0.2213],
         [ 0.8006,  0.4710,  0.1526,  ..., -0.4205,  0.2558,  0.0792],
         [ 0.8134,  0.3965,  0.0935,  ...,  0.2595,  0.0607, -0.3174]],

        [[-0.3465,  0.7238, -0.4847,  ..., -0.0640, -0.5665, -0.2855],
         [-0.1322,  1.1773, -0.4411,  ..., -0.1389, -0.3025,  0.1333],
         [-0.0498,  1.1296, -0.4354,  ..., -0.3101, -0.3020, -0.0058],
         ...,
         [-0.3358,  0.8488, -0.5039,  ...,  0.0633, -0.3793, -0.3095],
         [-0.3627,  0.8937, -0.4665,  ...,  0.0397, -0.3525, -0.2710],
         [-0.3758,  1.0932, -0.4787,  ..., -0.1589, -0.2444, -0.1691]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)
t

In [5]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(**inputs)
print(outputs)

print(outputs.logits)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.1431,  2.1121],
        [ 3.3840, -2.8064]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[-2.1431,  2.1121],
        [ 3.3840, -2.8064]], grad_fn=<AddmmBackward0>)


![Stage 3](image-3.png)

In [6]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim = -1)
print(predictions)

tensor([[0.0140, 0.9860],
        [0.9980, 0.0020]], grad_fn=<SoftmaxBackward0>)


In [7]:
model.config.id2label # First sentence positive, second sentence negative.

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [8]:
# You know 3 stage inside the pipeline now.

In [9]:
# Done !